In [ ]:
import numpy as np
from scipy.stats import erlang
from scipy.stats import norm

In [ ]:
# Initialization
N = 3
M = 3
h = 10 * np.ones((M+1, N))
H = [[100, 110, 120], [10, 20, 30], [10, 20, 30], [10, 20, 30]]
p = 100

shape = 1
scale = 1

# planned leadtimes
planned_leadtimes = np.zeros((M+1, N))
for i in range(M+1):
    for j in range(N):
        sample = erlang.rvs(shape, size=100)
        mean = np.mean(sample)
        std = np.std(sample)
        planned_leadtimes[i][j] = norm.ppf(0.8, loc = mean, scale = std)
print(planned_leadtimes)

# Simulation
iterations = 10000
holding_cost = np.zeros(iterations)
back_ordering_cost = np.zeros(iterations)
ontime_count = 0
for i in range(iterations):
    leadtimes = erlang.rvs(shape, size=(M+1, N))
    L = np.zeros(M)
    for j in range(M):
        for k in range(N):
            holding_cost[i] += H[j + 1][N - k - 1] * max(planned_leadtimes[j + 1][N - k - 1] - L[j], leadtimes[j + 1][N - k - 1])
            L[j] = max(0, L[j] + leadtimes[j + 1][N - k - 1] - planned_leadtimes[j + 1][N - k - 1])
    tardiness = max(L)
    for j in range(N):
        holding_cost[i] += H[0][N - k - 1] * max(planned_leadtimes[0][N - k - 1] - tardiness, leadtimes[0][N - k - 1])
        tardiness = max(0, tardiness + leadtimes[0][N - k - 1] - planned_leadtimes[0][N - k - 1])
    if tardiness < 1e-8:
        ontime_count += 1
    back_ordering_cost[i] = tardiness * p

[[1.80620023 2.04602466]
 [1.49122647 1.97524524]
 [1.90533722 1.91741929]
 [1.96354364 1.78701206]]


In [ ]:
print(np.mean(holding_cost))
print(np.mean(back_ordering_cost))
print(ontime_count / iterations)

528.0693486680372
52.91477097326889
0.746
